In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [2]:
sess = tf.InteractiveSession()
train_path = '/Users/apple/Desktop/Kaggle/DigitRecognizer/train.csv'
test_path = '/Users/apple/Desktop/Kaggle/DigitRecognizer/test.csv'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [3]:
train.shape

(42000, 785)

In [4]:
test.shape

(28000, 784)

In [95]:
scaler = MinMaxScaler()
X_train = train.drop('label',1)
X_train = scaler.fit_transform(X_train)
Y_train = train['label'].astype(np.float32)
Y_train = Y_train.reshape((-1,1))
X_test = test
Y_train = OneHotEncoder().fit_transform(Y_train).todense()
X_train[0]

/Library/Python/2.7/site-packages/ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """


array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [6]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

In [7]:
def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

In [8]:
def conv2d(x,W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

In [9]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [10]:
train.shape
test.shape

(28000, 784)

In [11]:
Y_train.shape

(42000, 10)

In [12]:
X_train.shape

(42000, 784)

In [93]:
h_pool2

<tf.Tensor 'MaxPool_1:0' shape=(?, 7, 7, 64) dtype=float32>

In [17]:
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
x_image = tf.reshape(x,[-1,28,28,1])

In [18]:
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [19]:
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [20]:
W_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)

In [21]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

In [22]:
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)

In [143]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_conv),reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [144]:
correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [145]:
X_train.shape

(42000, 784)

In [146]:
print X_train.shape

(42000, 784)


In [147]:
def get_batch(X,Y,step,batch_size):
    for i in range(step // batch_size):
        batch_x = X[i*batch_size:i*batch_size + batch_size]
        batch_y = Y[i*batch_size:i*batch_size + batch_size]
        yield batch_x,batch_y,i

In [148]:
Y_train[0:10]

matrix([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [150]:
tf.global_variables_initializer().run()
for bx,by,i in get_batch(X_train,Y_train,Y_train.shape[0],25):
    #print bx.shape ,by.shape
    train_step.run(feed_dict={x:bx,y:by,keep_prob:0.9})
    if i % 100 == 0:
        train_acc = accuracy.eval(feed_dict={x:bx,y:by,keep_prob:1.0})
        print 'step %d, trainging acc %g' %(i,train_acc)
#print accuracy.eval(feed_dict={x:X_train,y:Y_train,keep_prob:0.5})
    #train_step.run(feed_dict={x:batch_X,y:batch_Y,keep_prob:0.5})

step 0, trainging acc 0.2
step 100, trainging acc 0.92
step 200, trainging acc 0.84
step 300, trainging acc 0.96
step 400, trainging acc 0.96
step 500, trainging acc 0.92
step 600, trainging acc 1
step 700, trainging acc 1
step 800, trainging acc 0.96
step 900, trainging acc 1
step 1000, trainging acc 0.96
step 1100, trainging acc 0.96
step 1200, trainging acc 0.96
step 1300, trainging acc 0.96
step 1400, trainging acc 0.96
step 1500, trainging acc 1
step 1600, trainging acc 0.92


In [ ]:
modelPath = "/Users/apple/model.ckpt"
save = tf.train.Saver().save(sess,modelPath)
print accuracy.eval(feed_dict ={x:X_train,y:Y_train,keep_prob:1.0})

In [127]:
X_test_ = scaler.transform(X_test)
prediction = tf.argmax(y_conv,1)
result = []
for i in range(len(X_test_//50)):
    result.append(sess.run(prediction,feed_dict={x: X_test_[i*50:i*50+50],keep_prob:1.0}))

In [133]:
result1 = np.zeros(28000)
for i in range(result1.shape[0]):
    result1[i] = result[i//50][i%50]

In [134]:
cnn_sub = pd.DataFrame({'ImageId':range(1,28001),'Label':result1})

In [135]:
cnn_sub.to_csv('/Users/apple/Desktop/cnn_sub.csv')